In [1]:
import tangelo
from tangelo import SecondQuantizedMolecule
from tangelo.algorithms import ADAPTSolver,FCISolver
import matplotlib.pyplot as plt
import numpy as np
from tangelo import SecondQuantizedMolecule
from tangelo.algorithms import ADAPTSolver
from tangelo.toolboxes.qubit_mappings.mapping_transform import fermion_to_qubit_mapping
from tangelo.toolboxes.operators import count_qubits

In [2]:
H4 = [("H", (0, 0, 0)), ("H", (0, 1.4, 0)), ("H", (0, 2.8, 0)), ("H", (0, 4.2, 0))]
mol = SecondQuantizedMolecule(H4, q=0, spin=0, basis="sto-3g", frozen_orbitals=None)

opt_dict = {"molecule": mol, "tol": 0.01, "max_cycles": 7, "verbose": False, "qubit_mapping": "jw"}


fermion_operator = mol._get_fermionic_hamiltonian()
qubit_operator = fermion_to_qubit_mapping(fermion_operator, 'jw', mol.n_active_sos, mol.n_electrons)
n_qubits = count_qubits(qubit_operator)

In [3]:
def get_pool(qubit_hamiltonian, n_qubits):
    """Use Hamiltonian to identify non-commuting Pauli strings to use as operator pool.
    We identify terms with even number of Y-operators, in order to define excitations 
    which preserve T-reversal symmetry. We remove all Z operators, and we flip the first
    X or Y to its partner (i.e. X <> Y).
    Args:
        qubit_hamiltonian (QubitOperator): input Hamiltonian
        n_qubits (int): number of qubits for Hamiltonian
    
    Returns:
        pool_generators (list of QubitOperator): list of generators
    """
    import numpy as np
    from tangelo.toolboxes.operators.operators import QubitOperator

    pauli_lookup = {'Z':1, 'X':2, 'Y':3}
    pauli_reverse_lookup = ['I', 'Z', 'X', 'Y']
    pool_generators, pool_tuples = list(), list()
    indices = list()
    for term in qubit_hamiltonian.terms:
        pauli_string = np.zeros(n_qubits, dtype=int)
    
        #identify all X or Y factors
        for index, action in term:
            if pauli_lookup[action] > 1:
                pauli_string[index] = pauli_lookup[action]
        
           
        
        #only allow one operator acting on a given set of qubits
        action_mask = tuple(pauli_string > 1)
        print(f'term={term} pauli_string={pauli_string},action_mask={action_mask}')
        if action_mask in indices:
            continue

        #only consider terms with even number of Y
        if sum(pauli_string) % 2 == 0 and sum(pauli_string) > 0:
            #identify qubit operator to change X<>Y
            flip_index = np.where(pauli_string > 1)[0][0]
            pauli_string[flip_index] += (-1)**(pauli_string[flip_index] % 2) 
            

            #update set of used qubit combinations
            indices.append(action_mask)
            #create QubitOperator for the new generator
            operator_tuple = tuple([(index,pauli_reverse_lookup[pauli]) for index, pauli in enumerate(pauli_string) if pauli > 0])
            #We don't use the coefficients directly, so since we need to multiply by 1.j for evaluating gradients, 
            #I'm just instantiating these with that coefficient in place
            print(f'##flip_index={flip_index},pauli_string={pauli_string},indices={indices}')
            pool_generators.append(QubitOperator(operator_tuple, 1.0j))
            pool_tuples.append(operator_tuple)
    
    return pool_generators

In [6]:
qubit_pool = get_pool(qubit_hamiltonian=qubit_operator,n_qubits=n_qubits )

term=() pauli_string=[0 0 0 0 0 0 0 0],action_mask=(False, False, False, False, False, False, False, False)
term=((0, 'Z'),) pauli_string=[0 0 0 0 0 0 0 0],action_mask=(False, False, False, False, False, False, False, False)
term=((0, 'Y'), (1, 'Z'), (2, 'Z'), (3, 'Z'), (4, 'Y')) pauli_string=[3 0 0 0 3 0 0 0],action_mask=(True, False, False, False, True, False, False, False)
##flip_index=0,pauli_string=[2 0 0 0 3 0 0 0],indices=[(True, False, False, False, True, False, False, False)]
term=((0, 'X'), (1, 'Z'), (2, 'Z'), (3, 'Z'), (4, 'X')) pauli_string=[2 0 0 0 2 0 0 0],action_mask=(True, False, False, False, True, False, False, False)
term=((1, 'Z'),) pauli_string=[0 0 0 0 0 0 0 0],action_mask=(False, False, False, False, False, False, False, False)
term=((1, 'Y'), (2, 'Z'), (3, 'Z'), (4, 'Z'), (5, 'Y')) pauli_string=[0 3 0 0 0 3 0 0],action_mask=(False, True, False, False, False, True, False, False)
##flip_index=1,pauli_string=[0 2 0 0 0 3 0 0],indices=[(True, False, False, False, Tr

In [9]:
qubit_pool

[1j [X0 Y4],
 1j [X1 Y5],
 1j [X2 Y6],
 1j [X3 Y7],
 1j [X0 X1 X2 Y3],
 1j [X0 X1 X3 Y6],
 1j [X0 X1 X4 Y5],
 1j [X0 X1 X2 Y7],
 1j [X0 X1 X6 Y7],
 1j [X0 X2 X4 Y6],
 1j [X0 Y2 Y3 Y5],
 1j [X0 X3 X5 Y6],
 1j [X0 X3 X4 Y7],
 1j [X0 Y2 Y5 Y7],
 1j [X0 X5 X6 Y7],
 1j [X1 X2 X3 Y4],
 1j [X1 X2 X4 Y7],
 1j [X1 X2 X5 Y6],
 1j [X1 X3 X5 Y7],
 1j [X1 Y3 Y4 Y6],
 1j [X1 X4 X6 Y7],
 1j [X2 X3 X4 Y5],
 1j [X2 X3 X6 Y7],
 1j [X2 Y4 Y5 Y7],
 1j [X3 X4 X5 Y6],
 1j [X4 X5 X6 Y7]]

### Qubit-Adapt VQE

In [51]:
# from tangelo.toolboxes.operators import QubitOperator
# qubit_operator = QubitOperator(((0, 'X'), (1, 'X'), (2, 'Y'), (3, 'Y')), -1.0) \
#                   + QubitOperator(((0, 'X'), (1, 'Y'), (2, 'Y'), (3, 'X')), 1.0) \
#                   + QubitOperator(((0, 'Y'), (1, 'X'), (2, 'X'), (3, 'Y')), 1.0) \
#                   + QubitOperator(((0, 'Y'), (1, 'Y'), (2, 'X'), (3, 'X')), -1.0) 
pool_generators = get_pool(qubit_operator, n_qubits=4)
# print(f'OPERATOR POOL: {pool_generators}')

term=((0, 'X'), (1, 'X'), (2, 'Y'), (3, 'Y')) pauli_string=[2 2 3 3],action_mask=(True, True, True, True)
##flip_index=0,pauli_string=[3 2 3 3],indices=[(True, True, True, True)]
term=((0, 'X'), (1, 'Y'), (2, 'Y'), (3, 'X')) pauli_string=[2 3 3 2],action_mask=(True, True, True, True)
term=((0, 'Y'), (1, 'X'), (2, 'X'), (3, 'Y')) pauli_string=[3 2 2 3],action_mask=(True, True, True, True)
term=((0, 'Y'), (1, 'Y'), (2, 'X'), (3, 'X')) pauli_string=[3 3 2 2],action_mask=(True, True, True, True)


In [16]:

qubit_hamiltonian=qubit_operator
print(qubit_hamiltonian.terms)
n_qubits=4
pauli_lookup = {'Z': 1, 'X': 2, 'Y': 3}
pauli_reverse_lookup = ['I', 'Z', 'X', 'Y']
pool_generators, pool_tuples = list(), list()
indices = list()
for term in qubit_hamiltonian.terms:
    pauli_string = np.zeros(n_qubits, dtype=int)
    for index, action in term:
        print(f'index={index},action={action}')
        if pauli_lookup[action] > 1:
            pauli_string[index] = pauli_lookup[action]
    action_mask = tuple(pauli_string > 1)
print(pauli_string)


{((0, 'X'), (1, 'X'), (2, 'Y'), (3, 'Y')): -1.0, ((0, 'X'), (1, 'Y'), (2, 'Y'), (3, 'X')): 1.0, ((0, 'Y'), (1, 'X'), (2, 'X'), (3, 'Y')): 1.0, ((0, 'Y'), (1, 'Y'), (2, 'X'), (3, 'X')): -1.0}
index=0,action=X
index=1,action=X
index=2,action=Y
index=3,action=Y
index=0,action=X
index=1,action=Y
index=2,action=Y
index=3,action=X
index=0,action=Y
index=1,action=X
index=2,action=X
index=3,action=Y
index=0,action=Y
index=1,action=Y
index=2,action=X
index=3,action=X
[3 3 2 2]


In [19]:
action_mask = tuple(pauli_string > 1)

In [20]:
pauli_string

array([3, 3, 2, 2])

In [26]:
x = np.array([1,2,3,4])
tuple(x>1)

(False, True, True, True)

In [27]:
sum(pauli_string)

10

In [34]:
np.where(pauli_string > 1)[0][0]

2

In [31]:
pauli_string=np.array([0,1,2,3])